In [1]:
MARKET = 'SNP'

In [2]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [3]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder

In [4]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = DATAPATH.joinpath('temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [5]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

In [6]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
np.sort(np.array(files))

array(['df_chains.pkl', 'df_fresh.pkl', 'df_ohlcs.pkl', 'df_symlots.pkl',
       'df_und_margins.pkl', 'df_unds.pkl', 'dfrq.pkl', 'qopts.pkl',
       'z_new_qopts_temp.pkl'], dtype='<U20')

# Running `Covers`

In [7]:
from dfrq import get_dfrq
from support import Timer

In [8]:
ibp = Vars(MARKET.upper())  # IB Parameters from var.yml

HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

LOGPATH = pathlib.Path.cwd().joinpath(THIS_FOLDER, "data", "log")
DATAPATH = pathlib.Path.cwd().joinpath(THIS_FOLDER, "data", MARKET.lower())

# * SETUP LOGS AND CLEAR THEM
LOGFILE = LOGPATH.joinpath(MARKET.lower() + "_cover.log")
util.logToFile(path=LOGFILE, level=30)
with open(LOGFILE, "w"):
    pass

# . start the time
cover_time = Timer('cover')
cover_time.start()
dfrq = get_dfrq(MARKET)


cover started at 20-Nov-2020 21:20:46

dfrqs started at 20-Nov-2020 21:20:46

Done margin for ['AIG0115P35.0', 'NVDA1120C735.0'] 31 out of 59. Pending ['ADSK1127P175.0', 'ADSK1127C325.0']

...dfrqs took: 00:00:10 seconds



In [9]:
dfrq

,symbol,undPrice,lot,gross,grosspos,remq,status
16,TSLA,497.30,100,49927.0,264500.000000,0,harvest
13,NVDA,537.00,100,53761.0,147500.000000,0,harvest
50,AMGN,224.67,100,22467.0,92994.000240,0,uncovered
41,ADSK,256.88,100,25688.0,50000.000000,0,harvest
19,ALGN,441.35,100,44135.0,44095.700075,0,dodo
...,...,...,...,...,...,...,...
249,APA,11.35,100,1135.0,NaN,20,fresh
250,HPE,10.49,100,1049.0,NaN,21,fresh
251,GE,9.66,100,966.0,NaN,23,fresh
252,F,8.80,100,882.0,NaN,26,fresh


In [10]:
dfrq[dfrq.status.isin(['uncovered', 'dodo'])]

,symbol,undPrice,lot,gross,grosspos,remq,status
50,AMGN,224.67,100,22467.0,92994.000240,0,uncovered
19,ALGN,441.35,100,44135.0,44095.700075,0,dodo
20,LRCX,437.68,100,43768.0,43900.000000,0,dodo
29,UNH,337.00,100,33700.0,33589.999390,0,dodo
31,ANTM,311.85,100,31185.0,31184.997560,0,dodo
38,ANET,272.87,100,27287.0,27285.998535,0,dodo
94,ETSY,134.50,100,13450.0,27123.999020,0,dodo
74,UPS,166.48,100,16648.0,16711.999510,0,dodo
83,HSY,149.40,100,14940.0,14944.599915,1,dodo
151,RTX,70.87,100,7111.0,14087.999725,1,uncovered


In [11]:
from support import quick_pf, get_dte
from engine import executeAsync, post_df, price

with IB().connect(HOST, PORT, CID) as ib:
    df_pf = quick_pf(ib)

In [26]:
# ASSEMBLE STOCKS NEEDING COVER
# ... get the stocks and options
df_stk = df_pf[df_pf.secType == 'STK']
df_opt = df_pf[df_pf.secType == 'OPT']

# ... Filter out the partials
df_stk = df_stk[df_stk.position % 100 == 0]

# ... Filter out already covered and special stocks

#     ...from portfolio get symbol list of already existing covered calls
m_covered = (df_opt.position < 0) & df_opt.symbol.isin(df_stk.symbol)
already_covered = df_opt[m_covered].symbol.to_list()

df_uncovered = df_stk[~df_stk.symbol.isin(already_covered)]

# BUILD UP OPTIONS FOR THESE SYMBOLS

opts = [q for q in qopts if q.symbol in df_uncovered.symbol.unique()]
df_opts = util.df(opts)

optcols = "conId,symbol,secType,lastTradeDateOrContractMonth,strike,right".split(",")
df_uc = df_opts[optcols].rename(columns={"lastTradeDateOrContractMonth": "expiry"})

In [29]:
# ... get next week's earliest dte
df_uc = df_uc.assign(dte=df_uc.expiry.apply(get_dte))

m = (df_uc.dte>5)
df_uc = df_uc[m][df_uc[m].dte == df_uc[m].groupby('symbol').dte.transform(min)].reset_index(drop=True)

In [31]:
%%time
with IB().connect(HOST, PORT, CID) as ib:
    und_cts = ib.qualifyContracts(*[Contract(conId=c) for c in df_uncovered.conId])
    df_und_price = ib.run(
                    executeAsync(
                    ib=ib,
                    algo=price,
                    cts=und_cts,
                    CONCURRENT=500,
                    TIMEOUT=15,
                    post_process=post_df,
                    **{'FILL_DELAY': 15},
            )
        )


Done price for [] 0 out of 15. Pending ['ALGN0.0', 'AMGN0.0']
Wall time: 17.2 s


In [32]:
df_uncovered

,secType,conId,symbol,expiry,strike,right,position,mktPrice,mktVal,avgCost,unPnL,rePnL
6,STK,11459264,ALGN,,0.0,0,-100.0,440.957001,-44095.70,439.990157,-96.68,0.0
9,STK,266145,AMGN,,0.0,0,200.0,224.970001,44994.00,240.000000,-3006.00,0.0
11,STK,153912055,ANET,,0.0,0,-100.0,272.859985,-27286.00,229.994798,-4286.52,0.0
12,STK,173962302,ANTM,,0.0,0,-100.0,311.849976,-31185.00,302.493196,-935.68,0.0
13,STK,5407,CAH,,0.0,0,-100.0,52.283798,-5228.38,50.498765,-178.50,0.0
15,STK,5749,CL,,0.0,0,-100.0,85.370003,-8537.00,81.998069,-337.19,0.0
16,STK,13878278,CNC,,0.0,0,-100.0,63.610001,-6361.00,65.998422,238.84,0.0
23,STK,190480965,ETSY,,0.0,0,-200.0,135.619995,-27124.00,144.496688,1775.34,0.0
24,STK,98542021,EXPE,,0.0,0,-100.0,125.000000,-12500.00,119.997229,-500.28,0.0
28,STK,8199,HSY,,0.0,0,-100.0,149.445999,-14944.60,147.996610,-144.94,0.0


In [17]:
df_uc = df_uc.set_index('symbol')\
             .join(df_und_price.set_index('symbol')[['price', 'iv']])\
             .rename(columns={'iv': 'und_iv'})

In [22]:
df_uc = df_uc.join(df_uncovered.set_index('symbol')[['position', 'avgCost', 'unPnL']]).reset_index(drop=True)